# Tutorial

This tutorial takes you through the Flair library. 

## NLP base types

The Sentence object is the central object to our library. It holds a Sentence, consisting of Tokens. To this object, various layers of linguistic annotation may be added. This is also the central object for embedding your text.

Let's illustrate this with an example sentence.

In [4]:
# The sentence objects holds a sentence that we may want to embed
from flair.data import Sentence

# Make a sentence object by passing a whitespace tokenized string
sentence = Sentence('The grass is green .')

# Print the object to see what's in there
print(sentence)

Sentence: "The grass is green ." - 5 Tokens


Each word in a sentence is a Token object. You can directly access a token using the token_id. Each token has attributes, such as an id and a text.

In [6]:
print(sentence[4])

Token: 4 green


You can also iterate over all tokens in a sentence.

In [7]:
for token in sentence:
    print(token)

Token: 1 The
Token: 2 grass
Token: 3 is
Token: 4 green
Token: 5 .


Tokens can also have tags, such as a named entity tag. In this example, we're adding an NER tag of type 'color' to 
the word 'green' in the example sentence.


In [8]:
# add a tag to a word in the sentence
sentence[4].add_tag('ner', 'color')

# print the sentence with all tags of this type
print(sentence.to_ner_string())

The grass is green <color> .


## Tagging with Pre-Trained Models

Now, lets use a pre-trained model for named entity recognition (NER). 
This model was trained over the English CoNLL-03 task and can recognize 4 different entity
types.


In [21]:
from flair.tagging_model import SequenceTaggerLSTM

tagger = SequenceTaggerLSTM.load('ner')

All you need to do is use the `predict()` method of the tagger on a sentence. This will add predicted tags to the tokens
in the sentence. Lets use a sentence with two named
entities: 

In [12]:
sentence = Sentence('George Washington went to Washington .')

# predict NER tags
tagger.predict(sentence)

# print sentence with predicted tags
print(sentence.to_tag_string())

George <B-PER> Washington <E-PER> went to Washington <S-LOC> .


You chose which pre-trained model you load by passing the appropriate 
string you pass to the `load()` method of the `SequenceTaggerLSTM` class. Currently, the following pre-trained models
are provided (more coming): 
 

 'ner' : Conll-03 Named Entity Recognition (English)     

 'chunk' : Conll-2000 Syntactic Chunking (English)    


## Embeddings

We provide a set of classes with which you can embed the words in sentences in various ways. Note that all embedding 
classes inherit from the `TextEmbeddings` class and implement the `embed()` method which you need to call 
to embed your text. This means that for most users of Flair, the complexity of different embeddings remains hidden 
behind this interface. Simply instantiate the embedding class you require and call `embed()` to embed your text.

All embeddings produced with our methods are pytorch vectors, so they can be immediately used for training and 
fine-tuning.

### Classic Word Embeddings

Classic word embeddings are static and word-level, meaning that each distinc word gets exactly one pre-computed 
embedding. Most embeddings fall under this class, including the popular GloVe or Komnios embeddings. 

Simply instantiate the WordEmbeddings class and pass a string identifier of the embedding you wish to load. So, if 
you want to use GloVe embeddings, pass the string 'glove' to the constructor: 

In [16]:
# all embeddings inherit from the TextEmbeddings class. Init a simple glove embedding.
from flair.embeddings import WordEmbeddings
glove_embedding = WordEmbeddings('glove')

Now, create an example sentence and call the embedding's `embed()` method. You always pass a list of sentences to 
this method since some embedding types make use of batching to increase speed. So if you only have one sentence, 
pass a list containing only one sentence:


In [17]:
# embed a sentence using glove.
from flair.data import Sentence
sentence = Sentence('The grass is green .')
glove_embedding.embed(sentences=[sentence])

# now check out the embedded tokens.
for token in sentence:
    print(token)
    print(token.embedding.size())

Token: 1 The
torch.Size([100])
Token: 2 grass
torch.Size([100])
Token: 3 is
torch.Size([100])
Token: 4 green
torch.Size([100])
Token: 5 .
torch.Size([100])


This prints out the tokens and their embeddings. GloVe embeddings are pytorch vectors of dimensionality 100.

You choose which pre-trained embeddings you load by passing the appropriate 
string you pass to the constructor of the `WordEmbeddings` class. Currently, the following static embeddings
are provided (more coming): 
 
'glove' : GloVe embeddings 

'extvec' : Komnios embeddings 

'ft-crawl' : FastText embeddings 

'ft-german' : German FastText embeddings 

So, if you want to load German FastText embeddings, instantiate the method as follows:


In [18]:
german_embedding = WordEmbeddings('ft-german')

### Contextual String Embeddings


Contextual string embeddings are [powerful embeddings](https://drive.google.com/file/d/17yVpFA7MmXaQFTe-HDpZuqw9fJlmzg56/view?usp=sharing)
 that capture latent syntactic-semantic information that goes beyond
standard word embeddings. Key differences are: (1) they are trained without any explicit notion of words and
thus fundamentally model words as sequences of characters. And (2) they are **contextualized** by their
surrounding text, meaning that the *same word will have different embeddings depending on its
contextual use*.

With Flair, you can use these embeddings simply by instantiating the appropriate embedding class, same as before:

In [19]:
# the CharLMEmbedding also inherits from the TextEmbeddings class
from flair.embeddings import CharLMEmbeddings
charlm_embedding_forward = CharLMEmbeddings('news-forward')

# embed a sentence using CharLM.
from flair.data import Sentence
sentence = Sentence('The grass is green .')
charlm_embedding_forward.embed(sentences=[sentence])

FORWARD language mode loaded
on cuda:
False


You choose which embeddings you load by passing the appropriate 
string you pass to the constructor of the `CharLMEmbeddings` class. Currently, the following contextual string
 embeddings
are provided (more coming): 
 
| ID | Language | Embedding | 
| -------------     | ------------- | ------------- |
| 'news-forward'    | English | Forward LM embeddings over 1 billion word corpus |
| 'news-backward'   | English | Backward LM embeddings over 1 billion word corpus |
| 'mix-forward'     | English | Forward LM embeddings over mixed corpus (Web, Wikipedia, Subtitles) |
| 'mix-backward'    | English | Backward LM embeddings over mixed corpus (Web, Wikipedia, Subtitles) |
| 'german-forward'  | German  | Forward LM embeddings over mixed corpus (Web, Wikipedia, Subtitles) |
| 'german-backward' | German  | Backward LM embeddings over mixed corpus (Web, Wikipedia, Subtitles) |

So, if you want to load embeddings from the English news backward LM model, instantiate the method as follows:


In [20]:
charlm_embedding_backward = CharLMEmbeddings('news-backward')

BACKWARD language mode loaded
on cuda:
False


### Character Embeddings

Some embeddings - such as character-features - are not pre-trained but rather trained on the downstream task. Normally
this requires you to implement a [hierarchical embedding architecture](http://neuroner.com/NeuroNERengine_with_caption_no_figure.png). 

With Flair, you need not worry about such things. Just choose the appropriate
embedding class and character features will then automatically train during downstream task training. 


In [1]:
# the CharLMEmbedding also inherits from the TextEmbeddings class
from flair.embeddings import CharacterEmbeddings
embedder = CharacterEmbeddings()

# embed a sentence using CharLM.
from flair.data import Sentence
sentence = Sentence('The grass is green .')
embedder.embed(sentences=[sentence])


## Stacked Embeddings

Stacked embeddings are one of the most important concepts of this library. You can use them to combine different embeddings
together, for instance if you want to use both traditional embeddings together with contextual sting embeddings. 
Stacked embeddings allow you to mix and match. We find that a combination of embeddings often gives best results. 

All you need to do is use the `StackedEmbeddings` class and instantiate it by passing a list of embeddings that you wish 
to combine. For instance, lets combine classic GloVe embeddings with embeddings from a forward and backward 
character language model.

First, instantiate the three embeddings you wish to combine: 

In [2]:
# the CharLMEmbedding also inherits from the TextEmbeddings class
from flair.embeddings import WordEmbeddings, CharLMEmbeddings

# init GloVe embedding
glove_embedding = WordEmbeddings('glove')

# init CharLM embedding
charlm_embedding_forward = CharLMEmbeddings('news-forward')
charlm_embedding_backward = CharLMEmbeddings('news-backward')

FORWARD language mode loaded
on cuda:
False


BACKWARD language mode loaded
on cuda:
False


Now instantiate the `StackedEmbeddings` class and pass it a list containing these three embeddings.


In [3]:
# now create the StackedEmbedding object that combines all embeddings
from flair.embeddings import StackedEmbeddings
stacked_embeddings = StackedEmbeddings(embeddings=[glove_embedding, charlm_embedding_forward, charlm_embedding_backward])

That's it! Now just use this embedding like all the other embeddings, i.e. call the `embed()` method over your sentences.


In [4]:
# just embed a sentence using the StackedEmbedding as you would with any single embedding.
from flair.data import Sentence
sentence = Sentence('The grass is green .')
stacked_embeddings.embed(sentences=[sentence])

# now check out the embedded tokens.
for token in sentence:
    print(token)
    print(token.embedding)

Token: 1 The
tensor([-3.8194e-02, -2.4487e-01,  7.2812e-01,  ..., -2.5692e-05,
        -5.9604e-03, -2.5547e-03])
Token: 2 grass
tensor([-8.1353e-01,  9.4042e-01, -2.4048e-01,  ..., -6.7730e-05,
        -3.0360e-03, -1.3282e-02])
Token: 3 is
tensor([-5.4264e-01,  4.1476e-01,  1.0322e+00,  ..., -6.5714e-03,
        -3.5937e-03, -1.4478e-03])
Token: 4 green
tensor([-6.7907e-01,  3.4908e-01, -2.3984e-01,  ..., -2.2562e-05,
        -1.0895e-04, -4.3916e-03])
Token: 5 .
tensor([-3.3979e-01,  2.0941e-01,  4.6348e-01,  ...,  4.1382e-05,
        -4.4364e-04, -2.5425e-02])


Words are now embedding using a concatenation of three different embeddings. This means that the resulting embedding
vector is still a single Pytorch vector. 


## Training a Model

Here is example code for a small NER model trained over CoNLL-03 data, using simple GloVe embeddings.
In this example, we downsample the data to 10% of the original data. 
